# Multi-Agent Quickstart Tutorial

This quickstart tutorial demonstrates how to create a multi-agent system using the Atomic Agents library. The system will include a search agent and a calculator agent, which can be orchestrated to handle different types of user queries.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KennyVaneetvelde/atomic_agents/blob/main/examples/notebooks/multi_agent_quickstart.ipynb)


## Prerequisites

Before proceeding with this notebook, it is highly recommended to read up on the basics of the following libraries:

- **Pydantic**: A data validation and settings management library using Python type annotations. You can find more information and documentation at [Pydantic GitHub](https://github.com/pydantic/pydantic).
- **Instructor**: A Python library that simplifies working with structured outputs from large language models (LLMs). It provides a user-friendly API to manage validation, retries, and streaming responses. More details can be found at [Instructor GitHub](https://github.com/jxnl/instructor).

Understanding these libraries will help you make the most of this library.


## Install Necessary Packages

First, we need to install the required packages. Run the following command to install `atomic-agents`, `openai`, and `instructor` libraries.

In [ ]:
# Install necessary packages
%pip install atomic-agents openai instructor

## Import Libraries

We will import the necessary libraries for creating the multi-agent system.

In [ ]:
import os
from typing import Union
import instructor
import openai
from pydantic import create_model
from rich.console import Console
from atomic_agents.agents.base_agent import BaseIOSchema, BaseAgent, BaseAgentConfig
from atomic_agents.agents.tool_interface_agent import ToolInterfaceAgent, ToolInterfaceAgentConfig
from atomic_agents.lib.tools.search.searxng_tool import SearxNGTool, SearxNGToolConfig
from atomic_agents.lib.tools.calculator_tool import CalculatorTool, CalculatorToolConfig

## Initialize Components

Initialize the necessary components including the client, tools, and agents.

In [ ]:
# Initialize the console for output
console = Console()

# Initialize the client
client = instructor.from_openai(openai.OpenAI())

# Initialize the SearxNG search tool
searxng_tool = SearxNGTool(SearxNGToolConfig(base_url=os.getenv('SEARXNG_BASE_URL'), max_results=10))

# Initialize the calculator tool
calc_tool = CalculatorTool(CalculatorToolConfig())

## Configure and Create Agents

Configure and create the search agent and calculator agent.

In [ ]:
# Configure the search agent
search_agent_config = ToolInterfaceAgentConfig(client=client, model='gpt-4o-mini', tool_instance=searxng_tool, return_raw_output=False)

# Configure the calculator agent
calculator_agent_config = ToolInterfaceAgentConfig(client=client, model='gpt-4o-mini', tool_instance=calc_tool, return_raw_output=False)

# Create the search agent and calculator agent
searx_agent = ToolInterfaceAgent(config=search_agent_config)
calc_agent = ToolInterfaceAgent(config=calculator_agent_config)

## Define Union Response Schema

Create a union response schema that can handle responses from both the search agent and the calculator agent.

In [ ]:
# Define a union response schema
UnionResponse = create_model('UnionResponse', __base__=BaseIOSchema, response=(Union[searx_agent.input_schema, calc_agent.input_schema], ...))

Notice how instead of defining the response schema as a class, we use create_model() to define the schema. This is because we want to use the TOOL.input_schema to dynamically get the input schema of the agent, which is not possible with a class definition.

## Create Orchestration Agent

Create an orchestration agent that can manage the interactions between the user and the individual agents.

In [ ]:
# Create the orchestration agent
orchestration_agent = BaseAgent(config=BaseAgentConfig(client=client, model='gpt-4o-mini', output_schema=UnionResponse))

## Main Chat Loop

Create a main chat loop to interact with the orchestration agent. The agent will determine whether to use the search tool or the calculator tool based on the user's input.

In [ ]:
while True:
    user_input = input('You: ')
    if user_input.lower() in ['exit', 'quit']:
        print('Exiting chat...')
        break

    response = orchestration_agent.run(orchestration_agent.input_schema(chat_message=user_input))
    console.print(f'Agent: {response.response}')

    if isinstance(response.response, searx_agent.input_schema):
        console.print(f'Using searx agent')
        response = searx_agent.run(response.response)
    elif isinstance(response.response, calc_agent.input_schema):
        console.print(f'Using calc agent')
        response = calc_agent.run(response.response)

    console.print(f'Agent: {response.chat_message}')

## Conclusion

In this notebook, we demonstrated how to create a multi-agent system using the Atomic Agents library. The system includes a search agent and a calculator agent, which can be orchestrated to handle different types of user queries. You can further customize the agents and enhance their capabilities based on your requirements.

You can also create your own tools and wrap them into the tool agent. This allows you to extend the functionality of the multi-agent system to suit your specific needs.